# generate topology for harp

In [1]:
import gzip
import pickle
import torch
import numpy as np
import pickle 
import json

In [2]:
# the data should be put into the HARP codebase to run
root = 'home/user/HARP/'

In [3]:
# use the whole asn first
# this is for asn whole graph, you can also use asn with 98/510 nodes with node information saved in TE_TELGEN/raw/asn_graph
path = 'raw/asn_graph/ASN2k.json'
with open(path, 'r') as file:
    data = json.load(file)
data.keys()
# keys are same as harp

In [4]:
# same as harp (abilene, geant), no need to change
data['directed'], data['multigraph'], data['graph'], data['nodes'], data['links']

In [5]:
with open(root+'topologies/asn1739/t1.json', 'w') as file:
    json.dump(data, file)

In [6]:
# check saved asn topology
path = root+'topologies/asn1739/t1.json'
with open(path, 'r') as file:
    data = json.load(file)
data

# generate pairs, traffic matrices and paths

### Data Format:
#### 1) Traffic matrices: Numpy array of shape (num_pairs, 1)
#### 2) Pairs: Numpy array of shape (num_pairs, 2)
#### Note: the kth demand in the traffic matrix must correspond to the kth pair in the set of pairs file. This relation must be preserved for all snapshots. We suggest sorting the hash map (pairs/keys and values/demands) before separating.
#### 3) Paths: By default, HARP computes K shortest paths and automatically puts them in the correct folders and format.
#### If you wish to use your paths:
#### i) create a Python dictionary where the keys are the pairs and the values are a list of K lists, where the inner lists are a sequence of edges.
#### ii) For example: {(s, t): [[(s, a), (a, t)], [(s, a), (a, b), (b, t)]]}. 
#### iii) Put it inside topologies/paths_dict and name it: topo_name_K_paths_dict_cluster_num_cluster.pkl
###### For example: abilene_8_paths_dict_cluster_0.pkl
#### iiii) Make sure all pairs have the same number of paths (replicate if needed).

In [7]:
root = 'home/user/HARP/'

In [8]:
from itertools import islice
import networkx as nx


def k_shortest_paths(G, source, target, k, weight=None):
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))


f = open('raw/asn_graph/ASN2k.json')
G = json.load(f)

asn_graph = nx.DiGraph()
asn_graph.add_nodes_from([i['id'] for i in G['nodes']])
asn_graph.add_edges_from([(i['source'], i['target']) for i in G['links']])
print('Strongly connected:', nx.is_strongly_connected(asn_graph))
print('# of nodes and edges:', asn_graph.number_of_nodes(), asn_graph.number_of_edges())
print('Weighted:', nx.is_weighted(asn_graph))

In [9]:
# teal_harp_migrate: save the raw generated for teal and harp to migrate

file = open('raw/raw/teal_harp_migrate/instance_0_stds.pkl', 'rb')
std = pickle.load(file)

In [10]:
# create pairs data of harp
pairs = []
tms = []
paths = {}
for i in std:
    for j in i:
        pairs.append(j[0])
        tms.append(j[1])
        k_paths = k_shortest_paths(asn_graph, j[0][0], j[0][1], k=4)
        new_k_paths = []
        for p in k_paths:
            p = [(p[i], p[i+1]) for i in range(len(p)-1)]
            new_k_paths.append(p)
        paths[(j[0][0], j[0][1])] = new_k_paths
pairs = np.array(pairs).reshape(-1, 2)
tms = np.array(tms).reshape(-1, 1)

In [11]:
with open(root+'pairs/asn1739/t1.pkl', 'wb') as file:  
    pickle.dump(pairs, file)
with open(root+'traffic_matrices/asn1739/t1.pkl', 'wb') as file:  
    pickle.dump(tms, file)
with open(root+'topologies/paths_dict/asn1739_4_paths_dict_cluster_0.pkl', 'wb') as file:  
    pickle.dump(paths, file)

### pairs

In [12]:
# check saved pairs, tms, paths
file = open(root+'pairs/asn1739/t1.pkl', 'rb')
pairs = pickle.load(file)
pairs.shape, pairs

### traffic matrices

In [13]:
# check saved pairs, tms, paths
file = open(root+'traffic_matrices/asn1739/t1.pkl', 'rb')
tms = pickle.load(file)
tms.shape, tms

### paths

In [14]:
# check saved pairs, tms, paths
file = open(root+'topologies/paths_dict/asn1739_4_paths_dict_cluster_0.pkl', 'rb')
paths = pickle.load(file)
len(paths.keys()), paths